<a href="https://colab.research.google.com/github/Sanaghufan/Computer-Vision-and-Deep-Learning-Projects/blob/main/CropDisease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import os
os.environ["KAGGLE_USERNAME"] ="sana206"
os.environ["KAGGLE_KEY"] = "7aa7b4eb5c12f3b5324507ef68ac0dc8"
!kaggle datasets download vipoooool/new-plant-diseases-dataset

 99% 2.68G/2.70G [00:23<00:00, 140MB/s]
100% 2.70G/2.70G [00:23<00:00, 123MB/s]


In [ ]:

from IPython.display import clear_output
!unzip new-plant-diseases-dataset.zip
clear_output()

In [ ]:
%load_ext tensorboard

In [ ]:
import cv2
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Conv2D,MaxPool2D,Dense,Flatten,Dropout
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_dir = "new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/train"
test_dir = "test/test"
val_dir = "new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/valid"


In [ ]:
class_names = os.listdir(train_dir)
len(class_names)

38

In [ ]:
image_data_gen = ImageDataGenerator(rescale=1/255.,
                                    horizontal_flip=True)

X_train = image_data_gen.flow_from_directory(train_dir, target_size=(227,227),
                                             batch_size=32, class_mode="sparse")

X_val = image_data_gen.flow_from_directory(val_dir, target_size=(227,227),
                                             batch_size=32, class_mode="sparse")

X_test = image_data_gen.flow_from_directory(test_dir, target_size=(227,227),
                                             batch_size=32, class_mode="sparse")

Found 70295 images belonging to 38 classes.
Found 17572 images belonging to 38 classes.
Found 0 images belonging to 0 classes.


In [ ]:
from tensorflow.keras.applications import InceptionV3,VGG19,ResNet101
resnet = ResNet101()

179648224/179648224 [==============================] - 1s 0us/step


In [ ]:
resnet.summary()

Model: "resnet101"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                          

In [ ]:
resnet.output

<KerasTensor: shape=(None, 1000) dtype=float32 (created by layer 'predictions')>

In [ ]:
x = resnet.output
x=Dense(1024,activation = "relu")(x)
output = Dense(38,activation = "softmax")(x)

In [ ]:
model = Model(inputs = resnet.input,outputs = output)

In [ ]:
from tensorflow.python.ops import histogram_ops
from datetime import datetime as dt
def train_model():
  model.compile(optimizer="adam",
                loss="sparse_categorical_crossentropy",
                metrics=["accuracy"])

  logdir = os.path.join("logs", dt.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
  model.fit(X_train, validation_data=X_val, epochs=10, callbacks=[tensorboard_callback])


train_model()

Epoch 1/10
2197/2197 [==============================] - 1531s 656ms/step - loss: 1.8628 - accuracy: 0.4474 - val_loss: 0.7967 - val_accuracy: 0.7497
Epoch 2/10
2197/2197 [==============================] - 1439s 655ms/step - loss: 0.5219 - accuracy: 0.8385 - val_loss: 0.4519 - val_accuracy: 0.8629
Epoch 3/10
2197/2197 [==============================] - 1437s 654ms/step - loss: 0.2985 - accuracy: 0.9069 - val_loss: 0.2514 - val_accuracy: 0.9208
Epoch 4/10
2197/2197 [==============================] - 1435s 653ms/step - loss: 0.3088 - accuracy: 0.9066 - val_loss: 4.5692 - val_accuracy: 0.0646
Epoch 5/10
2197/2197 [==============================] - 1435s 653ms/step - loss: 0.2121 - accuracy: 0.9347 - val_loss: 0.1806 - val_accuracy: 0.9421
Epoch 6/10
2197/2197 [==============================] - 1435s 653ms/step - loss: 0.2195 - accuracy: 0.9323 - val_loss: 0.3382 - val_accuracy: 0.8984
Epoch 7/10
2197/2197 [==============================] - 1476s 672ms/step - loss: 0.1836 - accuracy: 0.9432